# Juan Manuel Deutsch, Cesar Felipe Giraldo, Julian Felipe Pulido

# Exercise 1

Para este ejercicio tenemos $X$ siendo la matriz base y $Σ$ como la matriz de covarianza de $X$

Tambien tenemos los componentes principales $u_1, u_2,...,u_n$, que son los autovectores o vectores propios de $Σ$ que tienen varianzas asociadas $u_i^TΣu_i$, donde $u_i^T$ es la transpuesta del vector propio $u_i$

Ahora tendremos una tranformación lineal de los datos $X$, siendo $AX$ donde $A$ es una transformación $m x m$

Ahora con la matriz transformada tenemos la matriz de covarianza transformada $AΣA^T$ cuyos componentes principales estan dados por $v_1, v_2,...,v_n$ estos siendo los autovectores cuyas varianza asociadas estan dadas por $v_i^TAΣA^Tv_i$

Teniendo en cuenta que necesitamos demostrar los componentes principales y como cambian tenemos con relacion a $AX$ y $X$

$u_1$ siendo componente principal cuya varianza asociada es $u_i^TΣu_i$ y siendo $v_1$ el componente principal cuya varianza asociada es $v_i^TAΣA^Tv_i$

Teniendo en cuenta que $u_1$ y $v_1$ estan relacionados por una transformación lineal tenemos que $v_1 = Tu_1$ para una matriz T

Ahora, se puede escribir la varianza asociada de $v_1$ como:

$$v_1^TAΣA^Tv_1 = (Tu_1)^TAΣA^T(Tu_1) = u_1^TT^TAΣA^TTu_1$$

Dado que $T$ es una matriz de transformacion lineal, $T^TAΣA^TT$ es otra matriz de covarianza $Σ'$

Por lo tanto ahora tenemos:

$$v_1^TAΣA^Tv_1 = u_1^TΣ'u_1$$

Concluyendo con esta demostración, tenemos que la varianza asociada $v_1$ depende de la matriz de covarianza $Σ'$ que es una version transformada de la matriz de covarianza original $Σ$, por lo cual, $Σ'$ es diferente a $Σ$, entonces la varianza asociada a estos van a ser diferentes, por lo que un valor propio esta ligado conjunto de datos original y el otro valor propio esta ligado al conjunto transformado

## 1.2

Para esta demostracion tenemos $D$, como una matriz $f x c$ donde $f$ son filas y $c$ son columnas, ademas de $D'$ que es la matriz transformada con una transformación afin a una o mas columnas

Iniciando calculamos la matriz de correlación $corr(D)$
$$corr(D) = 1/n (D - D_u)^T(D - D_u)$$
Donde $D_u$ es la media de cada columna de $D$
De acuerdo a eso se los componentes principales por medio de $corr(D)$

Ahora realizamos el mismo proceso con la tranformación afin, que seria del siguiente modo:

$$D' = A*D+b$$

Donde $A$ es la transformación lineal y $b$ es el desplazamiento

Ahora calculamos la correlacion 

$$corr(D') = 1/n (D' - D'_u)^T(D' - D'_u)$$

Donde ahora $D'_u$ es la media de la matriz transformada, donde las columnas de $D'$ se ven afectadas por la transformación afin

Pero debemos tener en cuenta que para la covarianza los efectos de una transformación afin se dan de la siguiente forma:

$$cov(D') = A^Tcov(D)A$$

No obstante, estamos tratando es con matrices de correlación, las cuales son versiones escaladas de las matrices de covariaza, por lo cual la matriz de covarianza tambien afecta a la matriz de correlación con la misma transformación afin

Por lo cual los autovectores o vectores propios de $corr(D')$ son los mismos que $corr(D)$

# Exercise 2

# 1

In [40]:

import numpy as np

def PCA(D, red):

    mu = np.mean(D, axis=0)
    Z = D - mu

    cov_matrix = np.cov(Z, rowvar=False, bias=False)
    eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
    
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[sorted_indices]
    
    if red < 1:
        total_variance = np.sum(eigenvalues)
        variance_top = red * total_variance
        cumulative_variance = 0
        r = 0

        for i in range(len(eigenvalues)):
            cumulative_variance += eigenvalues[i]
            r += 1
            if cumulative_variance > variance_top:
                break
    else:
        
        r = red
    
    top_eigenvectors = eigenvectors[:, :r] 
    total_variance = np.sum(eigenvalues[:r]) #luego lo usaremos en el B y el C
   
    variance = total_variance / np.sum(eigenvalues)
    
    A = np.dot(Z, top_eigenvectors)
    
    v = np.sum(eigenvalues[:r])
    
    return A, v

#Prueba

D = np.array([[1, 2, 3],
               [4, 5, 6],
               [7, 8, 9],
               [10, 11, 12],
               [13, 14, 15]])



A, variance = PCA(D, 2)
print(A)
print(variance)

[[ 1.03923048e+01 -1.55431223e-15]
 [ 5.19615242e+00 -7.77156117e-16]
 [ 0.00000000e+00  0.00000000e+00]
 [-5.19615242e+00  7.77156117e-16]
 [-1.03923048e+01  1.55431223e-15]]
67.5


# 2

In [59]:
import pandas as pd
import numpy as np
import time

def getStats(A):

        mu = list(sum(row)/len(row) for row in zip(*A))

        n = A.shape[0]
        cov_mu = A - mu
        cov = np.dot(cov_mu.T, cov_mu) / (n)
 
        desv = np.sqrt(np.diag(cov))
        cor = cov / np.outer(desv, desv)

        return mu, cov, cor

iris_data = pd.read_csv('iris.csv')
baseball_data = pd.read_csv('baseball_numeric.csv')
fabert_data = pd.read_csv('fabert.csv')
amazon_data = pd.read_csv('amazon.csv')


datasets = {'iris': iris_data, 'baseball_numeric': baseball_data, 'fabert': fabert_data}
results = []

for dataset_name, dataset in datasets.items():

    numeric_cols = dataset.select_dtypes(include=[np.number]).columns
    dataset_numeric = dataset[numeric_cols]
    
    A = dataset_numeric.values.astype(float)  
    
    start_time_getStats = time.time()
    mu, cov, cor = getStats(A)
    getStats_time = time.time() - start_time_getStats
    
    start_time_np = time.time()
    cov_np = np.cov(A, rowvar=False, bias=False)
    desv = np.sqrt(np.diag(cov_np))
    cor_np = cov_np / np.outer(desv, desv)
    np_time = time.time() - start_time_np
    
    results.append({'Dataset': dataset_name, 'Tiempo getStats': getStats_time, 'Tiempo Numpy': np_time})


results_df = pd.DataFrame(results)
print(results_df)

/var/folders/4k/4b01rf9j7cl6mcd3652m260w0000gn/T/ipykernel_83553/3864761767.py:14: RuntimeWarning: invalid value encountered in divide
  cor = cov / np.outer(desv, desv)


            Dataset  Tiempo getStats  Tiempo Numpy
0              iris         0.001135      0.001010
1  baseball_numeric         0.006199      0.000266
2            fabert         0.789193      0.399049


/var/folders/4k/4b01rf9j7cl6mcd3652m260w0000gn/T/ipykernel_83553/3864761767.py:41: RuntimeWarning: invalid value encountered in divide
  cor_np = cov_np / np.outer(desv, desv)


In [65]:
import timeit
import numpy as np
import pandas as pd

def getStats(A):
    start = timeit.default_timer()
    #Media
    mu = list(sum(row)/len(row) for row in zip(*A))
    #Covarianza
    n = A.shape[0]
    cov_mu = A - mu
    cov = np.dot(cov_mu.T, cov_mu) / (n)
    return mu, cov, timeit.default_timer() - start

def getStatsNumpy(A):
    start = timeit.default_timer()
    mu = np.mean(A, axis=0)
    cov = np.cov(A, rowvar=False, bias=False)
    return mu, cov, timeit.default_timer() - start

adf2 = amazon_data.drop('Class', axis=1).values
fdf2 = fabert_data.drop('class', axis=1).values
idf2 = iris_data.drop('species', axis=1).values
bndf = baseball_data.values

all_results = []
datasets = [adf2, bndf, fdf2, idf2]
data_dict = ['Amazon Dataset', 'Baseball Dataset', 'Fabert Dataset', 'Iris Dataset']

for dataset in datasets:
    combined_results = np.array([getStats(dataset)[2], getStatsNumpy(dataset)[2]]) 
    all_results.append(combined_results)

df = pd.DataFrame(all_results, index = data_dict, columns=['Nuestra función', 'Función de numpy'])
display(df) 

,Nuestra función,Función de numpy
Amazon Dataset,5.343213,4.389391
Baseball Dataset,0.007893,0.000376
Fabert Dataset,0.785908,0.154366
Iris Dataset,0.000145,0.000112


In [58]:
#B
import numpy as np
import pandas as pd

datasets = {'iris': iris_data, 'baseball_numeric': baseball_data, 'fabert': fabert_data, 'amazon': amazon_data}
results = []
for dataset_name, dataset in datasets.items():
    
    numeric_dataset = dataset.select_dtypes(include=[np.number])
    A = numeric_dataset.values.astype(float)

    reduced_data, variance = PCA(A, 0.9)
    
    total_variance_original = np.sum(np.linalg.eigvals(np.cov(A, rowvar=False)))
    relative_total_variance = variance / total_variance_original
    
    results.append({'Dataset': dataset_name, 'Dimensiones': reduced_data.shape[1], 'Varianza total absoluta': variance, 'Varianza Total Relativa': relative_total_variance})

results_df = pd.DataFrame(results)
print(results_df)

AttributeError: 'numpy.ndarray' object has no attribute 'select_dtypes'

In [18]:
#C
import numpy as np
import pandas as pd

datasets = {'iris': iris_data, 'baseball_numeric': baseball_data, 'fabert': fabert_data}
results = []

for dataset_name, dataset in datasets.items():

    numeric_dataset = dataset.select_dtypes(include=[np.number])
    
    X = numeric_dataset.values.astype(float)
    
    variances = []
    relative_variances = []
    
    for n_components in [1, 2, 3]:

        reduced_data, total_variance = PCA(X, n_components)
        relative_total_variance = total_variance / np.sum(np.linalg.eig(np.cov(X, rowvar=False, bias=False))[0])
        
        variances.append(total_variance)
        relative_variances.append(relative_total_variance)
    
    results.append({'Dataset': dataset_name, 'Varianza (1D)': variances[0], 'Varianza Relativa (1D)': relative_variances[0],
                    'Varianza (2D)': variances[1], 'Varianza Relativa(2D)': relative_variances[1],
                    'Varianza (3D)': variances[2], 'Varianza Relativa (3D)': relative_variances[2]})

results_df = pd.DataFrame(results)
results_df = results_df.round(3)  
print(results_df)

1
[[ 0.36158968]
 [-0.08226889]
 [ 0.85657211]
 [ 0.35884393]]
2
[[ 0.36158968 -0.65653988]
 [-0.08226889 -0.72971237]
 [ 0.85657211  0.1757674 ]
 [ 0.35884393  0.07470647]]
3
[[ 0.36158968 -0.65653988 -0.58099728]
 [-0.08226889 -0.72971237  0.59641809]
 [ 0.85657211  0.1757674   0.07252408]
 [ 0.35884393  0.07470647  0.54906091]]
1
[[ 4.55662820e-05]
 [-7.50537856e-05]
 [ 1.40999914e-05]
 [ 1.80208904e-04]
 [-1.68872525e-05]
 [-1.28535774e-06]
 [-5.33799512e-06]
 [ 9.91505066e-05]
 [ 2.34646730e-04]
 [ 1.18112857e-04]
 [-1.78881466e-04]
 [ 8.85085097e-05]
 [-1.20545727e-05]
 [-2.96589208e-04]
 [-4.47373039e-06]
 [ 8.20791589e-05]
 [ 6.67353669e-05]
 [-2.01096290e-04]
 [ 5.58729392e-05]
 [ 2.33363682e-05]
 [-4.06090246e-05]
 [-9.26691866e-05]
 [-1.29227378e-04]
 [ 9.29537526e-05]
 [-9.75742271e-05]
 [-1.02114640e-04]
 [-5.05601478e-06]
 [ 2.16900399e-04]
 [ 1.02012357e-04]
 [ 5.95708330e-07]
 [ 3.87984491e-05]
 [ 7.80143469e-05]
 [ 4.80646033e-05]
 [-1.11657006e-04]
 [-1.10023537e-04]
